In [1]:
!pip install pytorch_lightning --quiet
!pip install transformers --quiet
!pip install datasets --quiet

In [2]:
!pip install seqeval==0.0.2 --quiet

In [3]:
import pytorch_lightning as pl
from transformers import AutoTokenizer
import random
from typing import List
import h5py
import pytorch_lightning as pl
import torch
import torchmetrics
from pytorch_lightning import LightningDataModule, seed_everything, LightningModule, Trainer
from pytorch_lightning.callbacks.finetuning import BaseFinetuning
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from torch import nn
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim.optimizer import Optimizer
from torch.utils.data import Dataset, DataLoader
from transformers import AutoConfig, AutoModelForTokenClassification
# from seqeval.metrics import f1_score, precision_score, recall_score,accuracy_score
import numpy as np
import seqeval
seed_everything(123, workers=True)

INFO:lightning_lite.utilities.seed:Global seed set to 123


123

In [4]:
import os
import csv
import pandas as pd
from collections import defaultdict
from sklearn.utils import shuffle
import random
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def entities(entity_file):
  abs=[]
  chemical=[]
  disease=[]
  sentence_chem=[]
  sentence_dis=[]
  label=[]
  sentence=[]
  with open(entity_file) as txt_f:
    reader=csv.reader(txt_f, delimiter="\t")
    for row in reader:
      if len(row)==1:
        if ("|a|" in row[0] ) == True:
          abs.append(sent_tokenize((row[0].split("|a|"))[1]))
      
      if len(row)==6:
        if row[-2]=='Chemical':
          chemical.append(row[-3])
        elif row[-2]=='Disease':
          disease.append(row[-3])
  labels_to_ids= {
    'Oth': 0,
    'Chemical': 1,
    'Disease': 2
  }
  for i in abs:
    for j in i:
      l=re.sub('[^a-zA-Z]',' ',j)
      l=l.lower()
      l=l.split()
      l=[word for word in l if not word in stopwords.words('english')]
      sentence.append(l)
      l=word_tokenize(j)
      labs=[]
      for k in l:
        if k in chemical:
          labs.append('Chemical')
        elif k in disease:
          labs.append('Disease')
        else:
          labs.append('Oth')
      # label.append(' '.join(labs))
      label.append([labels_to_ids[i] for i in labs])

  f={
     'text':sentence,
     'labels':label
    }
  df=pd.DataFrame(f)
  return df

In [6]:
train_file='/content/CDR_TrainingSet.PubTator.txt'
train_df=entities(train_file)
test_file='/content/CDR_TestSet.PubTator.txt'
test_df=entities(test_file)
valid_file='/content/CDR_DevelopmentSet.PubTator.txt'
val_df=entities(valid_file)

In [ ]:
train_df.head()

,text,labels
0,"[unanesthetized, spontaneously, hypertensive, ...","[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[hypotensive, effect, mg, kg, alpha, methyldop...","[0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
2,"[naloxone, alone, affect, either, blood, press...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[brain, membranes, spontaneously, hypertensive...","[0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[findings, indicate, spontaneously, hypertensi...","[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
train_df.to_csv('cdr_train.csv',index=False)

In [ ]:
test_df.to_csv('cdr_test.csv',index=False)

In [ ]:
val_df.to_csv('cdr_val.csv',index=False)

In [7]:
def chemprot(abs,ent):
  
  with open(abs) as ab:
    abstract=csv.reader(ab, delimiter="\t")
    pmid_ab=[]
    abs=[]
    for i in abstract:
      pmid_ab.append(i[0])
      abs.append(i[1:])
  
  with open(ent) as et:
    entities=csv.reader(et,delimiter='\t')
    pmid_en=[]
    ner=[]
    name=[]
    gene=[]
    chemical=[]
    for j in entities:
      if j[2]=='CHEMICAL':
        chemical.append(j[5])
      else:
        gene.append(j[5])
  
  text=[]
  labels=[]
  labels_to_ids= {
    'Oth': 0,
    'Chemical': 1,
    'Gene': 3
  }
  for i in range(len(abs)):
    for j in range(len(abs[i])):
      l1=sent_tokenize(abs[i][j])
      for y in l1:
        l=re.sub('[^a-zA-Z]',' ',y)
        l=l.lower()
        l=l.split()
        l=[word for word in l if not word in stopwords.words('english')]
        l3=[]
        for m in range(len(l)):   
            if l[m] in chemical:
              l3.append('Chemical')
            elif l[m] in gene:
              l3.append('Gene')
            else:
              l3.append('Oth')
        if ('Chemical' or 'Gene') in l3:
          text.append(l)
          # labels.append(l3)
          labels.append([labels_to_ids[i] for i in l3]) 
  f={
     'text':text,
     'labels':labels
    }
  df=pd.DataFrame(f)
  return df  


In [8]:
tr_abs='/content/chemprot_training_abstracts.tsv'
tr_ent='/content/chemprot_training_entities.tsv'
train=chemprot(tr_abs,tr_ent)


In [ ]:
train_df.head()

,text,labels
0,"[unanesthetized, spontaneously, hypertensive, ...","[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[hypotensive, effect, mg, kg, alpha, methyldop...","[0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
2,"[naloxone, alone, affect, either, blood, press...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[brain, membranes, spontaneously, hypertensive...","[0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[findings, indicate, spontaneously, hypertensi...","[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
train.to_csv('chemprot_train.csv',index=False)

In [9]:
te_abs='/content/chemprot_test_abstracts_gs.tsv'
te_ent='/content/chemprot_test_entities_gs.tsv'
test=chemprot(te_abs,te_ent)


In [ ]:
test.to_csv('chemprot_test.csv',index=False)

In [11]:
de_abs='/content/chemprot_development_abstracts.tsv'
de_ent='/content/chemprot_development_entities.tsv'
val=chemprot(de_abs,de_ent)

In [ ]:
val.head()

,text,labels
0,"[binding, dimemorfan, sigma, receptor, anticon...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]"
1,"[dextromethorphan, methoxy, n, methylmorphinan...","[1, 1, 0, 0, 0, 0, 0, 0, 0]"
2,"[mechanisms, cns, effects, dm, suggested, asso...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
3,"[dm, largely, demethylated, phencyclidine, pcp...","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[dimemorfan, methyl, n, methylmorphinan, df, a...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
val.to_csv('chemprot_val.csv',index=False)

In [37]:
train1 = train_df.append(train, ignore_index=True)

In [ ]:
train1.to_csv('train.csv',index=False)

In [12]:
train1.columns

Index(['text', 'labels'], dtype='object')

In [13]:
train1.shape

(9167, 2)

In [38]:
val1=val_df.append(val,ignore_index=True)

In [ ]:
val1.to_csv('val.csv',index=False)

In [15]:
val1.columns

Index(['text', 'labels'], dtype='object')

In [39]:
test1=test_df.append(test,ignore_index=True)

In [ ]:
test1.to_csv('test.csv',index=False)

In [17]:
test1.columns

Index(['text', 'labels'], dtype='object')

In [18]:
train1=train_df[:100]
test1=test_df[:100]
val1=val_df[:100]

In [19]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1",num_labels=4)

In [20]:
label_names=[0,1,2,3]

In [21]:
input=tokenizer(train1['text'][0],is_split_into_words=True,max_length=128, truncation=True,padding="max_length")

In [ ]:
input.tokens()

In [23]:
input.word_ids

<bound method BatchEncoding.word_ids of {'input_ids': [101, 8362, 6354, 2050, 4638, 26300, 20061, 1193, 177, 24312, 27291, 13475, 9711, 1892, 2997, 1762, 2603, 1666, 1107, 4487, 7912, 2285, 172, 4934, 2386, 2042, 17599, 12139, 1116, 4023, 1107, 23034, 1174, 11802, 9468, 2858, 19315, 17713, 4023, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1

In [24]:
def align_labels_with_tokens(labels, word_ids):
  
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = 0 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(0)
        else:
            # Same word as previous token
            label = labels[word_id]
            new_labels.append(label)

    return new_labels

In [25]:
labels = train_df['labels'][0]
word_ids = input.word_ids()
print(labels)
print(len(align_labels_with_tokens(labels, word_ids)))

[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
128


In [26]:
def tokenize_and_align_labels(df):
  label_ids=[]
  input_ids=[]
  attention_mask=[]
  token_type_ids=[]

  for i in range(len(df['text'])):
      tokenized_inputs = tokenizer(
          df['text'][i], truncation=True, is_split_into_words=True,max_length=256,padding="max_length"
      )
      input_ids.append(tokenized_inputs['input_ids'])
      attention_mask.append(tokenized_inputs['attention_mask'])
      token_type_ids.append(tokenized_inputs['token_type_ids'])
      all_labels = df['labels'][i]
     
      word_ids = tokenized_inputs.word_ids()
      label_ids.append(align_labels_with_tokens(all_labels, word_ids))

      return input_ids,attention_mask,token_type_ids,label_ids

In [27]:
df=train_df[:1]

In [28]:
d=tokenize_and_align_labels(df)
# d[2]

In [29]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.text[index]  
        word_labels = self.data.labels[index]  
        tokenized_inputs = self.tokenizer(
          sentence, truncation=True, is_split_into_words=True,max_length=self.max_len,padding="max_length")
        word_ids = tokenized_inputs.word_ids()
        label_ids=align_labels_with_tokens(word_labels, word_ids)
        input_ids=(tokenized_inputs['input_ids'])
        attention_mask=(tokenized_inputs['attention_mask'])
        token_type_ids=(tokenized_inputs['token_type_ids'])
        return input_ids, attention_mask,token_type_ids, label_ids
    
    def __len__(self):
        return self.len

In [30]:
def my_collate(batch):
    input_ids = np.asarray([item[0] for item in batch])
    attention_mask = np.asarray([item[1] for item in batch])
    token_type_ids = np.asarray([item[2] for item in batch])
    label_ids = np.asarray([item[3] for item in batch])
    return torch.LongTensor(input_ids), torch.LongTensor(attention_mask),torch.LongTensor(token_type_ids),torch.LongTensor(label_ids)

class TransformerDataModule(LightningDataModule):
    def __init__(self,train_df,val_df,test_df, tokenizer,max_len=128,batch_size: int = 16):
        super().__init__()
        self.val_df=val_df
        self.test_df=test_df
        self.train_df=train_df
        self.batch_size = batch_size
        self.valid_dataset = None
        self.train_dataset = None
        self.test_dataset=None
        self.tokenizer=tokenizer
        self.max_len=max_len

    def prepare_data(self, *args, **kwargs):
        pass

    def setup(self, **kwargs):
        """define train and validation datasets """
        self.train_dataset = dataset(self.train_df,self.tokenizer,self.max_len)
        self.valid_dataset = dataset(self.val_df,self.tokenizer,self.max_len)
        self.test_dataset = dataset(self.test_df,self.tokenizer,self.max_len)

    def train_dataloader(self):
        """returns train dataloader"""
        loader = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True,
                            num_workers=2, pin_memory=True, collate_fn = my_collate)
        return loader
    
    def val_dataloader(self):
        """returns val dataloader"""
        loader = DataLoader(self.valid_dataset, batch_size=self.batch_size, shuffle=False,
                            num_workers=2, pin_memory=True, collate_fn = my_collate)
        return loader

    def test_dataloader(self):
        loader = DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False,
                            num_workers=2, pin_memory=True, collate_fn = my_collate)
        return loader



In [40]:
data_module = TransformerDataModule(
  train1,
  val1,
  test1,
  tokenizer,
  batch_size=8
)

In [32]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
import statistics
from statistics import mean

In [33]:

class MilestonesFinetuning(BaseFinetuning):
    def __init__(self, milestones: List = [50], train_bn: bool = False):
        super().__init__()
        self.milestones = milestones
        self.train_bn = train_bn

    def freeze_before_training(self, pl_module: pl.LightningModule):
        self.freeze(modules=pl_module.transformer.bert, train_bn=self.train_bn)

    def finetune_function(self, pl_module: pl.LightningModule, epoch: int, optimizer: Optimizer, opt_idx: int):
        if epoch == self.milestones[0]:
            print('starting model training')
            self.unfreeze_and_add_param_group(
                modules=pl_module.transformer.bert, optimizer=optimizer, train_bn=self.train_bn
            )
            
class TransformerModel(LightningModule):
    
    def __init__(self, lr=1e-3, lr_scheduler_gamma=1e-1, milestones=[50]):
        super(TransformerModel, self).__init__()

        self.lr = lr
        self.train_f1_score=[]
        self.val_f1_score=[]
        self.test_f1_score=[]
        self.lr_scheduler_gamma = lr_scheduler_gamma
        self.milestones = milestones
        self.labels = ['Oth','Chemical','Disease','Gene']
        self.label_map={i: label for i, label in enumerate(self.labels)}
        self.num_labels = len(self.labels)
        self.config = AutoConfig.from_pretrained('dmis-lab/biobert-large-cased-v1.1',
                num_labels=self.num_labels,
                id2label=self.label_map,
                label2id={label: i for i, label in enumerate(self.labels)})
        self.transformer = AutoModelForTokenClassification.from_pretrained('dmis-lab/biobert-large-cased-v1.1',config =self.config)
        self.save_hyperparameters("lr","lr_scheduler_gamma")
        self.binarizer = MultiLabelBinarizer()

    def align_predictions(self, predictions: np.ndarray, label_ids: np.ndarray):
        preds = predictions.argmax(2)

        batch_size, seq_len = preds.shape

        out_label_list = [[] for _ in range(batch_size)]
        preds_list = [[] for _ in range(batch_size)]
        
        for i in range(batch_size):
            for j in range(seq_len):
                if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                    out_label_list[i].append(self.label_map[int(label_ids[i][j])])
                    preds_list[i].append(self.label_map[int(preds[i][j])])
        return out_label_list, preds_list 

    def forward(self, input_ids, attention_mask, token_type_ids, label_ids):
        outputs = self.transformer(input_ids,attention_mask=attention_mask, token_type_ids=token_type_ids,labels = label_ids)
        return outputs

    def configure_optimizers(self):
        parameters = list(self.parameters())
        trainable_parameters = list(filter(lambda p: p.requires_grad, parameters))
        print(
            f"The model will start training with only {len(trainable_parameters)} "
            f"trainable parameters out of {len(parameters)}."
        )
        optimizer = torch.optim.AdamW(trainable_parameters, lr=self.lr)
        scheduler = MultiStepLR(optimizer, milestones=self.milestones, gamma=self.lr_scheduler_gamma)
        return [optimizer], [scheduler]
      
    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, token_type_ids, label_ids = batch
        outputs = self(input_ids, attention_mask, token_type_ids, label_ids)
        valid_loss = outputs[0]
        logits  = outputs[1]
        y_true, y_pred = self.align_predictions(logits,label_ids)
        self.binarizer.fit(y_true)
        self.val_f1_score.append(f1_score(self.binarizer.transform(y_true), 
                self.binarizer.transform(y_pred), average='weighted'))
        self.log('valid_f1',mean(self.val_f1_score), on_step=False, on_epoch=True, prog_bar=False, logger=True)
        # self.log('val_f1',,on_epoch=False, on_step=True, prog_bar=True,
        #          logger=True)
        # output_labels, predictions = self.align_predictions(logits,label_ids)
        # self.binarizer.fit(output_labels)

        # self.log('valid_loss', valid_loss, on_step=True, on_epoch=False, prog_bar=False, logger=True)
        # self.log('valid_f1',f1_score(self.binarizer.transform(output_labels), 
        #         self.binarizer.transform(predictions), average='weighted'),on_epoch=False, on_step=True, prog_bar=False,
        #          logger=True)
        # self.log('valid_precision', precision_score(output_labels, predictions),on_epoch=True, on_step=False, prog_bar=True,
        #          logger=True)
        # self.log('valid_recall', recall_score(output_labels, predictions),on_epoch=True, on_step=False, prog_bar=True,
        #          logger=True)

        
    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, token_type_ids, label_ids = batch
        outputs = self(input_ids, attention_mask, token_type_ids, label_ids)
        train_loss = outputs[0]
        logits  = outputs[1]
        y_true, y_pred = self.align_predictions(logits,label_ids)
        self.binarizer.fit(y_true)
        self.train_f1_score.append(f1_score(self.binarizer.transform(y_true), 
                self.binarizer.transform(y_pred), average='weighted'))
        self.log('train_f1',mean(self.train_f1_score),on_epoch=False, on_step=True, prog_bar=True,
                 logger=True)
        self.log('train_loss', train_loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        return train_loss

    

    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, token_type_ids, label_ids = batch
        outputs = self(input_ids, attention_mask, token_type_ids, label_ids)
        test_loss = outputs[0]
        logits  = outputs[1]
        output_labels, predictions = self.align_predictions(logits,label_ids)
        self.log('test_loss', test_loss, on_step=True, on_epoch=False, prog_bar=False, logger=True)
        self.log('test_f1', f1_score(output_labels, predictions),on_epoch=True, on_step=False, prog_bar=True,
                 logger=True)
        # self.log('test_precision', precision_score(output_labels, predictions),on_epoch=True, on_step=False, prog_bar=True,
        #          logger=True)
        # self.log('test_recall', recall_score(output_labels, predictions),on_epoch=True, on_step=False, prog_bar=True,
        #          logger=True)
        
        return test_loss

In [42]:
model = TransformerModel()

Some weights of the model checkpoint at dmis-lab/biobert-large-cased-v1.1 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not init

In [43]:
trainer = Trainer(max_epochs=1, gpus=1,
                  callbacks=[ModelCheckpoint(monitor='valid_f1', mode='max'), MilestonesFinetuning()])

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [44]:
trainer.fit(model,data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type                       | Params
-----------------------------------------------------------
0 | transformer | BertForTokenClassification | 363 M 
-----------------------------------------------------------
4.1 K     Trainable params
363 M     Non-trainable params
363 M     Total params
1,453.015 Total estimated model params size (MB)


The model will start training with only 2 trainable parameters out of 391.


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Gene'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Gene'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Gene'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Disease'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Gene'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Gene'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Gene'] will be ignored
  "unknown class(es) {0} will be igno

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Gene'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Chemical'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Gene'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Gene'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Chemical'] will be ignored
  "unknown class(es) {0} will be

In [50]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [51]:
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-large-cased-v1.1') 
model = TransformerModel.load_from_checkpoint('/content/gdrive/MyDrive/epoch=2-step=13752.ckpt')

Some weights of the model checkpoint at dmis-lab/biobert-large-cased-v1.1 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not init

In [53]:
labels = ['Oth','Chemical','Disease','Gene']
label_map={i: label for i, label in enumerate(labels)}

In [54]:
def get_predictions(sentence):
  tok = tokenizer(sentence, padding = 'max_length', max_length = 128)
  tokens = tokenizer.tokenize(sentence)
  input_ids, attention_mask, token_type_ids = tok['input_ids'], tok['attention_mask'],tok['token_type_ids']
  token_type_ids[0] = 1
  output = model(torch.LongTensor(input_ids).unsqueeze(0), torch.LongTensor(attention_mask).unsqueeze(0), torch.LongTensor(token_type_ids).unsqueeze(0), torch.LongTensor([0 for i in range(128)]).unsqueeze(0))[1]
  labels_out = output.argmax(2).squeeze(0).numpy().tolist()[1:len(tokens)+1]
  entities = [label_map[i] for i in labels_out]
  new_tokens, new_labels = [], []
  # final_tokens, final_labels = [], []
  for token, label_idx in zip(tokens, entities):
      if token.startswith("##"):
          new_tokens[-1] = new_tokens[-1] + token[2:]
      else:
          new_labels.append(label_idx)
          new_tokens.append(token)
  return new_tokens,new_labels

In [57]:
abstract='Desipramine treatment decreases 3H-nisoxetine binding and norepinephrine transporter mRNA in SK-N-SHSY5Y cells.	The antidepressant desipramine has been shown to decrease synaptic membrane concentrations of the norepinephrine re-uptake transporter (NET) in vivo and in vitro, on both an acute and a chronic basis. The possible contribution of decreased NET synthesis to the chronic downregulation of the NETs has not been definitively established. In this study, we treated SK-N-SHSY5Y cells with 100 nM desipramine for 24 or 72 h, and measured 3H-nisoxetine binding (as an estimate of NETs) and NET mRNA by quantitative reverse transcription polymerase chain reaction. Similar to what has been reported previously, membrane 3H-nisoxetine binding was significantly decreased at both 24 and 72 h (approximately 50% at both time points). However, a significant decrease (64 +/- 8% of paired control) of NET mRNA was observed only at the 72-h time point. We conclude that decreased NET synthesis may contribute to the chronic, but not acute, effect of desipramine to downregulate the NET.'
l = sent_tokenize(abstract)
for sentence in l:
  li=re.sub('[^a-zA-Z]',' ',sentence)
  li=li.lower()
  li=li.split()
  li=[word for word in li if not word in stopwords.words('english')]
  sentence = ' '.join(li)
  ner_tokens,ner_labels = get_predictions(sentence)

  for token, label in zip(ner_tokens, ner_labels):
      print("{}\t{}".format(label, token))

Chemical	desipramine
Chemical	treatment
Oth	decreases
Chemical	h
Oth	nisoxetine
Oth	binding
Chemical	norepinephrine
Oth	transporter
Oth	mrna
Oth	sk
Oth	n
Oth	shsy
Oth	cells
Chemical	antidepressant
Chemical	desipramine
Oth	shown
Oth	decrease
Oth	synaptic
Oth	membrane
Oth	concentrations
Chemical	norepinephrine
Oth	uptake
Oth	transporter
Oth	net
Oth	vivo
Oth	vitro
Oth	acute
Oth	chronic
Oth	basis
Oth	possible
Oth	contribution
Oth	decreased
Oth	net
Oth	synthesis
Oth	chronic
Oth	downregulation
Oth	nets
Oth	definitively
Oth	established
Oth	study
Oth	treated
Oth	sk
Oth	n
Oth	shsy
Oth	cells
Oth	nm
Chemical	desipramine
Oth	h
Oth	measured
Oth	h
Chemical	nisoxetine
Oth	binding
Oth	estimate
Oth	nets
Oth	net
Oth	mrna
Oth	quantitative
Oth	reverse
Oth	transcription
Oth	polymerase
Oth	chain
Oth	reaction
Oth	similar
Oth	reported
Oth	previously
Oth	membrane
Chemical	h
Oth	nisoxetine
Oth	binding
Chemical	significantly
Oth	decreased
Oth	h
Oth	approximately
Oth	time
Oth	points
Oth	however
Oth	significant
Ot